# Final Year Project
## Project # 3
### Group 3

In [56]:
import afunctions as func
import matplotlib.pyplot as plt
import skimage

In [57]:
IMG_PATH = r'C:/Users/ginof/OneDrive - ITU/Documentos/GitHub/fyp2021p3g03/data/example_image/'
SEG_PATH = r'C:/Users/ginof/OneDrive - ITU/Documentos/GitHub/fyp2021p3g03/data/example_segmentation/'

TRUTH = 'fyp2021p3g03/data/example_ground_truth.csv'
FEATURES = 'fyp2021p3g03/features/features.csv'

In [58]:
image = plt.imread(SEG_PATH+'ISIC_0001769_segmentation.png')

In [60]:
func.zoom(image)

NameError: name 'np' is not defined